In [26]:
import time
import csv
from IPython.display import display, Markdown, Latex
import networkx as nx
import bravo.regulation as regulation
import bravo.signaling as signaling
import bravo.config as config
import bravo.util as util
import pyBravo as bravo_main

def read_input_genes(filename):
    res = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in reader:
            res.append(''.join(row))
    return res

# Upstream regulation network reconstruction

The following lines of code specify that regulation will be expored at depth 3 (`config.MAX_DEPTH = 3`), on all datasources except *mirtarbase* (`config.DATA_SOURCES = set(ds) - set(['mirtarbase'])`), on a set of 3 genes `['SCN3A', 'SCN5A', 'HEY2']`. 

The regulation graph reconstruction is called with 
```
reconstructed_network = regulation.upstream_regulation(gene_list, 
                                                       already_explored = [], 
                                                       sif_network = [], 
                                                       current_depth = 0, 
                                                       explored_reg = 0)
```

Parameters are set to default values to allow re-execution of this cell without side-effect of previous executions. 

In [86]:
start_time = time.time()

# global options
config.MAX_DEPTH = 3
config.FAST = True
config.DECOMPOSE_COMPLEXES = True
config.EXTEND_WITH_SYNONYMS = True
config.EXTEND_WITH_SUFFIXES = True
config.VERBOSE = False

""" all possible data sources """
ds = ['bind', 'biogrid', 'corum',
                'ctd', 'dip', 'drugbank', 'hprd', 'humancyc', 'inoh',
                'intact', 'kegg', 'mirtarbase', 'netpath', 'panther',
                'pid', 'psp', 'reactome', 'reconx', 'smpdb', 'wp',
                'intact_complex', 'msigdb']

""" removing mirtarbase """
config.DATA_SOURCES = set(ds) - set(['mirtarbase'])

""" specifiyng inputs """
gene_list = ['SCN3A', 'SCN5A', 'HEY2']
#gene_list = read_input_genes('iLiverCancer1715.target')

""" reconstruction call """
reconstructed_network = regulation.upstream_regulation(gene_list, 
                                                       already_explored = [], 
                                                       sif_network = [], 
                                                       current_depth = 0, 
                                                       explored_reg = 0)

elapsed_time = round((time.time() - start_time), 2)

print("--- Upstream regulation network in %s seconds ---" % elapsed_time)


exploration depth 0
to be explored ['SCN3A', 'SCN5A', 'HEY2']

to be explored after complex decomposition ['SCN3A', 'SCN5A', 'HEY2']
new synonmys to be explored:['EIEE62', 'FFEVF4', 'NAC3', 'Nav1.3', 'CDCD2', 'CMD1E', 'CMPD2', 'HB1', 'HB2', 'HBBD', 'HH1', 'ICCD', 'IVF', 'LQT3', 'Nav1.5', 'PFHB1', 'SSS1', 'VF1', 'CHF1', 'GRIDLOCK', 'GRL', 'HERP1', 'HESR2', 'HRT2', 'bHLHb32']
exploring ['SCN3A', 'SCN5A', 'HEY2', 'EIEE62', 'FFEVF4', 'NAC3', 'Nav1.3', 'CDCD2', 'CMD1E', 'CMPD2', 'HB1', 'HB2', 'HBBD', 'HH1', 'ICCD', 'IVF', 'LQT3', 'Nav1.5', 'PFHB1', 'SSS1', 'VF1', 'CHF1', 'GRIDLOCK', 'GRL', 'HERP1', 'HESR2', 'HRT2', 'bHLHb32', 'SCN3A mRna', 'SCN3A protein', 'SCN5A mRna', 'SCN5A protein', 'HEY2 mRna', 'HEY2 protein', 'EIEE62 mRna', 'EIEE62 protein', 'FFEVF4 mRna', 'FFEVF4 protein', 'NAC3 mRna', 'NAC3 protein', 'Nav1.3 mRna', 'Nav1.3 protein', 'CDCD2 mRna', 'CDCD2 protein', 'CMD1E mRna', 'CMD1E protein', 'CMPD2 mRna', 'CMPD2 protein', 'HB1 mRna', 'HB1 protein']

Explored 2 regulators
explorin

## Metrics and SIF export of the regulation network

In [87]:
G = bravo_main.build_nx_digraph(reconstructed_network)

Number of nodes = 5
Number of edges = 4


In [88]:
G_unified = util.fast_reg_network_unification(G, util.index_syn)
print('Nodes after synonym-based unification = ' + str(len(G_unified.nodes())))
print('Edges after synonym-based unification = ' + str(len(G_unified.edges())))

Nodes after synonym-based unification = 5
Edges after synonym-based unification = 4


In [89]:
md = bravo_main.get_centrality_as_md(G_unified)
display(Markdown(md))


| Node | Degree Centrality |
|------|------|
| RBPJ/NICD | 0.75 | 
| HEY2 | 0.5 | 
| NICD | 0.25 | 
| RBPJ | 0.25 | 
| TAp73a (tetramer) | 0.25 | 


In [84]:
bravo_main.write_to_SIF(G_unified, 'tutorial-regulation.sif')
bravo_main.write_provenance(G_unified, 'tutorial-regulation-prov.csv')

SIF network written to tutorial-regulation.sif
Basic regulation reaction provenance written to tutorial-regulation-prov.csv


---

# Upstream signaling network reconstruction

These lines would test the regulation network reconstruction on all data sources, and stop at depth 4.  
```
config.MAX_DEPTH = 4
config.DATA_SOURCES = ['ctd', 'pid']
```

```
reconstructed_network = bravo.upstream_signaling(['SCN5A', 'SCN3A', 'HEY2'], 
        already_explored = [], 
        sif_network = [], 
        current_depth = 0, 
        explored_reg = 0)
)
```


In [113]:
start_time = time.time()

# global options
config.MAX_DEPTH = 4
config.DECOMPOSE_COMPLEXES = True
config.EXTEND_WITH_SYNONYMS = True
config.EXTEND_WITH_SUFFIXES = True
config.FINE_GRAINED_SIGNALING_SIF = False
config.FAST = True
config.VERBOSE = False

""" all possible data sources """
ds = ['bind', 'biogrid', 'corum',
                'ctd', 'dip', 'drugbank', 'hprd', 'humancyc', 'inoh',
                'intact', 'kegg', 'mirtarbase', 'netpath', 'panther',
                'pid', 'psp', 'reactome', 'reconx', 'smpdb', 'wp',
                'intact_complex', 'msigdb']

config.DATA_SOURCES = ds

""" reconstruction call """
reconstructed_network = signaling.upstream_signaling(['SCN5A', 'SCN3A', 'HEY2'], 
                                                     already_explored = [], 
                                                     sif_network = [], 
                                                     current_depth = 0, 
                                                     explored_reg = 0)

elapsed_time = round((time.time() - start_time), 2)
print("--- Upstream signaling network assembled in %s seconds ---" % elapsed_time)


exploration depth 0
to be explored after complex decomposition ['SCN5A', 'SCN3A', 'HEY2']
new synonmys to be explored:['CDCD2', 'CMD1E', 'CMPD2', 'HB1', 'HB2', 'HBBD', 'HH1', 'ICCD', 'IVF', 'LQT3', 'Nav1.5', 'PFHB1', 'SSS1', 'VF1', 'EIEE62', 'FFEVF4', 'NAC3', 'Nav1.3', 'CHF1', 'GRIDLOCK', 'GRL', 'HERP1', 'HESR2', 'HRT2', 'bHLHb32']

Explored 4 regulators

exploration depth 1
to be explored after complex decomposition ['NOTCH3 coactivator complex', 'Fyn', 'CAMK2D', 'PKACA']
new synonmys to be explored:['CAMKD', 'PRKACA', 'PPNAD4']

Explored 23 regulators

exploration depth 2
netrin 1/DCC/FAK/Fyn decomposed into ['netrin 1', 'DCC', 'FAK', 'Fyn']
CSK/PAG1 decomposed into ['CSK', 'PAG1']
netrin 1/UNC5B decomposed into ['netrin 1', 'UNC5B']
Tie2/Ang2 decomposed into ['Tie2', 'Ang2']
Ephrin A5/EPHA5 decomposed into ['Ephrin A5', 'EPHA5']
RHOA/GTP decomposed into ['RHOA', 'GTP']
Nephrin/NEPH1/podocin decomposed into ['Nephrin', 'NEPH1', 'podocin']
Complex (Sema3A/Neuropilin 1/ PlexinA1) deco


Explored 479 regulators

Explored 561 regulators

Explored 645 regulators

Explored 677 regulators

Explored 699 regulators

Explored 872 regulators

Explored 893 regulators

Explored 902 regulators

Explored 902 regulators

Explored 905 regulators

Explored 906 regulators

Explored 909 regulators

Explored 931 regulators

Explored 936 regulators

Explored 939 regulators

Explored 940 regulators
Exploring alted due to maximum depth
--- Upstream signaling network assembled in 10.6 seconds ---


In [114]:
G = bravo_main.build_nx_digraph(reconstructed_network)

Number of nodes = 1084
Number of edges = 1691


In [115]:
G_unified = util.fast_reg_network_unification(G, util.index_syn)
print('Nodes after synonym-based unification = ' + str(len(G_unified.nodes())))
print('Edges after synonym-based unification = ' + str(len(G_unified.edges())))

		merging node SHP1 into node NR0B2
		merging node DARPP-32 into node PPP1R1B
		merging node PRKACA into node PRKCA
		merging node PKCA into node PRKCA
		merging node PRKACA into node PRKCA
		merging node PDK1 into node PDPK1
		merging node FAK into node PTK2
		merging node NEPH1 into node KIRREL1
		merging node PI3K into node PIK3CG
		merging node ERK1 into node MAPK3
		merging node Yes into node YES1
		merging node Met into node SLTM
		merging node ERK2 into node MAPK1
		merging node ERK2 into node MAPK1
		merging node PKG1 into node PRKG1
		merging node cGK 1 into node PRKG1
		merging node NDK 7 into node NME7
		merging node NDK 6 into node NME6
		merging node PKD into node PRKD1
		merging node NDK 6 into node NME6
		merging node NDK 7 into node NME7
		merging node PKM2 into node PKM
		merging node DBL into node MCF2
		merging node FAK1 into node PTK2
		merging node DARPP-32 into node PPP1R1B
		merging node CaMK IV into node CAMK4
		merging node FAK1 into node PTK2
		merging node IK

In [116]:
md = bravo_main.get_centrality_as_md(G_unified)
display(Markdown(md))


| Node | Degree Centrality |
|------|------|
| Ca2+ | 0.159 | 
| PIK3CG | 0.082 | 
| Akt1 | 0.073 | 
| RAC1/GTP | 0.072 | 
| Src | 0.065 | 
| RHOA/GTP | 0.064 | 
| GTP | 0.059 | 
| PTK2 | 0.055 | 
| ERK1-2-active | 0.047 | 
| EGFR | 0.032 | 


In [106]:
bravo_main.write_to_SIF(G_unified, 'tutorial-signaling.sif')
bravo_main.write_provenance(G_unified, 'tutorial-signaling-prov.csv')

SIF network written to tutorial-signaling.sif
Basic regulation reaction provenance written to tutorial-signaling-prov.csv
